This notebook implements the Iterative Phase Estimation algorithm described in https://www.youtube.com/watch?v=aLSM0_H8hUE

In [31]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from pytket.circuit import CircBox, QControlBox, DiagonalBox
from pytket.extensions.qiskit import AerStateBackend
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from pytket.passes import DecomposeBoxes
import math
from datetime import datetime

In [37]:
"""
m: integer number of repetitions
omega: amount of Z rotation applied
state_prep_circuit: Circuit that prepares an eigenvector of unitary_circuit
unitary_circuit: circuit implementing a black box unitary
"""
def build_phase_estimation_circuit(
    m: int, omega: float, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", 1)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Add Hadamard gate to the measurement register
    qpe_circ.H(measurement_register[0])
    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
    # Run the controlled unitary m times
    for _ in range(m):
        qpe_circ.add_qcontrolbox(
            controlled_u_gate, list(measurement_register) + list(state_prep_register)
        )

    qpe_circ.U1(omega/np.pi, measurement_register[0])
    qpe_circ.H(measurement_register[0])
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [38]:
"""
runs phase estimation given the parameters, and then returns which outcome was most frequent
out of n_shots
"""
def get_next_digit(m: int, omega: float, state_prep_circuit: Circuit, unitary_circuit: Circuit,
                   backend=AerStateBackend(), n_shots=100):
    qpe_circ = build_phase_estimation_circuit(m, omega, state_prep_circuit, unitary_circuit)
    DecomposeBoxes().apply(qpe_circ)
    compiled_circ = backend.get_compiled_circuit(qpe_circ)
    result = backend.run_circuit(compiled_circ, n_shots)
    sorted_shots = result.get_counts().most_common()
    print(sorted_shots)
    return sorted_shots[0][0][0] 

In [39]:
"""
iterates on qpe, updating the algorithm adaptively
"""
def iterated_qpe(state_prep_circuit: Circuit, unitary_circuit: Circuit, precision: int, backend=AerStateBackend()):
    digits = []
    for i in range(precision-1,-1,-1):
        omega = 0
        for j in range(len(digits)):
            omega = omega - np.pi*digits[j]/2**(j+1)
        x = get_next_digit(2**i, omega, state_prep_circuit, unitary_circuit, backend=backend)
        digits.insert(0,x)
    bitstring = "".join([str(bit) for bit in digits])
    print(bitstring)
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

In [40]:
### SIMPLE EXAMPLE
input_angle = 0.52536912 # set this to what you please
true_phase = input_angle/2

prep_circuit = Circuit(1).X(0)  # prepare the |1> eigenstate of U1
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops

estimated_phase = iterated_qpe(prep_circuit, unitary_circuit, precision=7)

print("Estimated Phase:", estimated_phase)
print("True Phase:", true_phase)

error = round(abs(input_angle - (2 * estimated_phase)), 3)
print("Error:",error)

[((0,), 63), ((1,), 37)]
[((1,), 93), ((0,), 7)]
[((0,), 99), ((1,), 1)]
[((0,), 99), ((1,), 1)]
[((0,), 100)]
[((1,), 99), ((0,), 1)]
[((0,), 100)]
0100010
Estimated Phase: 0.265625
True Phase: 0.26268456
Error: 0.006


In [41]:
### Harmonic Oscillator Benchmarking
p = 2
precision = 3
iterative_phase_est = Nexus().new_project(f"Iterative Phase Estimation - Harmonic Oscillator - p: {p}, prec: {precision}, {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1E")
quantinuum_backend = NexusBackend(
    backend_config= configuration, 
    project= iterative_phase_est
)

def create_diagonal_arr(n):
    N = int(math.pow(2, n))
    return [np.exp(2*math.pi*1j*theta/N) for theta in range(0, N)]
prep_circuit = Circuit(p)

diagonal_box = DiagonalBox(create_diagonal_arr(p))
unitary_circuit = Circuit(p).add_diagonal_box(diagonal_box, range(p))

estimated_phase = iterated_qpe(prep_circuit, unitary_circuit, precision=precision,backend = quantinuum_backend)

print("Estimated Phase:", estimated_phase)
print("True Phase:", 0)


New project created: Iterative Phase Estimation - p: 2, prec: 3, 2024-02-04 07:10:00.653023

Started using project with name: Iterative Phase Estimation - p: 2, prec: 3, 2024-02-04 07:10:00.653023
[((0,), 90), ((1,), 10)]
[((0,), 95), ((1,), 5)]
[((0,), 99), ((1,), 1)]
000
Estimated Phase: 0.0
True Phase: 0


In [ ]:
### H2 Benchmarking
from pytket.circuit import QControlBox, CircBox, StatePreparationBox
from pytket.utils import QubitPauliOperator, gen_term_sequence_circuit
from pytket.pauli import Pauli, QubitPauliString

precision = 6
state = 0 # which state of the H2 molecule we are calculating for. 0 is the ground state, can choose from [0, 1, 2, 3]
iterative_phase_est = Nexus().new_project(f"Iterative Phase Estimation - H2 - statenum: {state}, prec: {precision}, {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1E")
quantinuum_backend = NexusBackend(
    backend_config= configuration, 
    project= iterative_phase_est
)

def build_u_t_approximation() -> Circuit:
    # t is pi/2
    # 1 Trotter step
    
    z1 = QubitPauliString({Qubit(0):Pauli.Z})
    z2 = QubitPauliString({Qubit(1):Pauli.Z})
    I =  QubitPauliString({Qubit(0):Pauli.I, Qubit(1):Pauli.I})
    zz = QubitPauliString({Qubit(0):Pauli.Z, Qubit(1):Pauli.Z})
    yy = QubitPauliString({Qubit(0):Pauli.Y, Qubit(1):Pauli.Y})

    H = QubitPauliOperator({z1: -0.3980, z2: - 0.3980, yy: - 0.1809, zz:0.0112, I:-0.3322})

    circ = gen_term_sequence_circuit(H, Circuit(2))
        
    return circ

# circuit to prepare the eigenstate of H for the specified state
prep_circuit = Circuit(2)
if state == 0:
    U = StatePreparationBox(np.array([ -.99376431765,0,0,.11150103564])) #ground eigenstate
    prep_circuit.add_state_preparation_box(U, [0,1])
    true_EV = -1.1373
    #eigenvalue is -1.1373
elif state == 1:
    prep_circuit.H(0)
    prep_circuit.CX(0,1)
    prep_circuit.X(1)
    true_EV = -0.5243
    #eigenvalue is -0.5243
elif state == 2:
    prep_circuit.H(0)
    prep_circuit.CX(0,1)
    prep_circuit.X(1)
    prep_circuit.Z(1)
    true_EV = -0.1625
    #eigenvalue is -0.1625
else:
    U= StatePreparationBox(np.array([.11150103564, 0,0, .99376431765])) #most excited eigenstate
    prep_circuit.add_state_preparation_box(U, [0,1])
    true_EV = 0.495297
    #eigenvalue is 0.495297

# Base unitary for controlled U ops
unitary_circuit = build_u_t_approximation()

estimated_phase = iterated_qpe(prep_circuit, unitary_circuit, precision=precision,backend = quantinuum_backend)

print("Estimated Phase:", estimated_phase*-4)
print("True Phase:", true_EV)


New project created: Iterative Phase Estimation - H2 - statenum: 0, prec: 6, 2024-02-04 07:30:07.415602

Started using project with name: Iterative Phase Estimation - H2 - statenum: 0, prec: 6, 2024-02-04 07:30:07.415602
